In [2]:
#from gx import *
import pymysql
import pymysql.cursors

def insert_weakly_connected(list, session="CANONICAL", erase_session=False, start_id=0):
    """ (list,   session, erase_session, start_id)
    
    
        list __ list of weakly connected digraphs
        
        optional arguments:
            
            session         text string identifying this session
            erase_session   set to true in order to drop all elements from this session
            start_id        starting graph id
    """
    mysql = pymysql.connect(host="127.0.0.1",user="digraphs",password="xxxxxxxxxxx",use_unicode=True,database="digraphs")
    cur = mysql.cursor()
    if erase_session:
        print "Erasing session data..."
        cur.execute("DELETE FROM `weakly_connected` WHERE SESSION=%s", (session,))
        mysql.commit()
    id = start_id
    for D in list:
        arglist = []
        cur = mysql.cursor()
        for u,v,lbl in D.edges():
            arglist.append((session,int(id),int(u),int(v)))
        retry = True
        while retry:
            try:
                cur.executemany("INSERT INTO `weakly_connected` (`SESSION`,`ID`,`U`,`V`) VALUES (%s,%s,%s,%s);",arglist)
                retry = False
            except Exception,e:
                retry = True
                print e
                print "Retrying..."
                mysql.commit()
                mysql.close()
                mysql = pymysql.connect(host="127.0.0.1",user="digraphs",password="d1gr4phs",use_unicode=True,database="digraphs")
                cur = mysql.cursor()
                
                
        id += 1
    mysql.commit()  
    mysql.close()

<p>In order to obtain all digraphs up to isomorphism with a bounded number of arcs; that have a single (weakly)-connected component; we start with the canonically labelled digraph with one arc, and then keep on adding arcs. All digraphs are then converted to canonically labelled.</p>
<p> </p>
<p><span style="font-family: courier new, courier;">encountered_digraphs</span> is a list of canoncially labelled digraphs with the above properties;</p>
<p><span style="font-family: courier new, courier;">current_head</span> contains pairs <span style="font-family: courier new, courier;">(D,n)</span> where <span style="font-family: courier new, courier;">D</span> is a canonically labelled digraph and <span style="font-family: courier new, courier;">n</span> is the maximal integer of any vertex of a new arc.</p>

In [3]:
D0 = PathDigraph([0,1]).canonical_label().copy(immutable=True)
encountered_digraphs = [D0]
current_head = [(D0,2)]
maxArcs = 10
insert_weakly_connected(encountered_digraphs,start_id=0, erase_session=False) #set to True to have a fresh start!

<p>Now let's iterate....</p>

In [6]:
for nArcs in range(2,maxArcs+1):
    print "nArcs =",nArcs
    new_head = set()
    encounters = 0
    id0 = len(encountered_digraphs)
    count0 = 0
    count1 = len(current_head)
    for D0, vMax in current_head:
        count0 += 1
        if count0 % 1000 == 0:
            print count0, "of",count1," = ",float(count0*100)/count1,"%"
        for u in range(vMax+1):
            for v in range(vMax+1):
                if u == v:
                    continue #no loops
                if D0.has_vertex(u) and D0.has_vertex(v) and D0.has_edge(u,v):
                    continue #not a new arc
                D1 = D0.copy(immutable=False)
                D1.add_edge(u,v)
                D1 = D1.canonical_label()
                encounters += 1
                new_head.add( (D1.copy(immutable=True),max([u+1,v+1,vMax])) )
    current_head = list(new_head)
    new_digraphs = [x[0] for x in current_head]
    print "Inserting into database... startid=",id0, " lastid=",id0+len(new_digraphs)-1
    insert_weakly_connected(new_digraphs,start_id=id0, erase_session=False)
    id0 += len(new_digraphs)
    encountered_digraphs.extend(new_digraphs)
    print "Total:", len(encountered_digraphs), "Head Size: ",len(current_head)," Avg.Multiplicity of Discovery:", float(encounters) / len(current_head)